In [2]:
import Pkg; Pkg.add("Optim")
import Pkg; Pkg.add("CSV")
import Pkg; Pkg.add("DataFrames")
import Pkg; Pkg.add("PyPlot")
import Pkg; Pkg.add("LaTeXStrings")
import Pkg; Pkg.add("Statisitics")

  Updating registry at `C:\Users\Rachel\.julia\registries\General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
Fetching: [========================================>]  99.9 %0.0 % [=>                                       ]  0.6 %3 % %5.2 %5.3 % [===>                                     ]  5.4 % %>                                     ]  6.2 %]  9.0 %0 %Fetching: [======>                                  ]  13.1 %                                  ]  13.3 %Fetching: [=======>                                 ]  16.7 %Fetching: [========>                                ]  17.7 %=========>                               ]  22.5 %                              ]  23.5 %==========>                              ]  24.6 %Fetching: [============>                            ]  27.6 % [============>                            ]  29.5 %                           ]  30.7 %Fetching: [==============>                          ]  32.7 %Fetching: [==============>                     

  [39dd38d3] + Dierckx v0.4.1
  [c5cfe0b6] - DiffBase v0.3.2
  [2b5f629d] ↑ DiffEqBase v4.32.0 ⇒ v5.7.0
  [eb300fae] ↑ DiffEqBiological v3.5.0 ⇒ v3.7.2
  [f3b72e0c] ↑ DiffEqDevTools v2.6.1 ⇒ v2.7.2
  [01453d9d] ↑ DiffEqDiffTools v0.7.1 ⇒ v0.8.1
  [c894b116] ↑ DiffEqJump v5.8.0 ⇒ v6.1.1
  [77a26b50] ↑ DiffEqNoiseProcess v2.4.2 ⇒ v3.1.0
  [1130ab10] ↑ DiffEqParamEstim v1.5.1 ⇒ v1.6.0
  [41bf760c] ↑ DiffEqSensitivity v3.0.0 ⇒ v3.2.2
  [163ba53b] ↑ DiffResults v0.0.3 ⇒ v0.0.4
  [b552c78f] ↑ DiffRules v0.0.7 ⇒ v0.0.10
  [b4f34e82] ↑ Distances v0.7.4 ⇒ v0.8.0
  [31c24e10] ↑ Distributions v0.16.4 ⇒ v0.18.0
  [ffbed154] + DocStringExtensions v0.7.0
  [da5c29d0] ↑ EllipsisNotation v0.3.0 ⇒ v0.4.0
  [d4d017d3] ↑ ExponentialUtilities v1.3.0 ⇒ v1.4.0
  [1a297f60] ↑ FillArrays v0.4.0 ⇒ v0.6.1
  [587475ba] ↑ Flux v0.7.1 ⇒ v0.8.3
  [f6369f11] ↑ ForwardDiff v0.10.2 ⇒ v0.10.3
  [38e38edf] + GLM v1.1.1
  [28b8d3ca] ↑ GR v0.37.0 ⇒ v0.39.1
  [42fd0dbc] + IterativeSolvers v0.8.1
  [82899510] ↑ IteratorInte

In [1]:
using Optim
using CSV
using DataFrames
using PyPlot
using LaTeXStrings
using Printf
using LinearAlgebra
using Statistics

In [2]:
df_henry = CSV.read("../data/henry_constants.csv")

,crystal,CO2_henry coefficient [mmol/(g-bar)],CO2_Temperature [K],SO2_henry coefficient [mmol/(g-bar)]
,String⍰,Float64⍰,Int64⍰,Float64⍰
1,MFM601,1.3703,298,97.2407
2,Zn-bdc-ted,2.15867,298,19.1696
3,Ni-bdc-ted,2.2069,298,40.402
4,NOTT-202a,3.27573,293,153.6
5,MFM600,1.41883,298,34.7973
6,NOTT-300,7.96526,293,483.864
7,MFM-300-In,4.60751,298,171.432
8,Zn-CN6,5.17058,298,4.36483
9,Mg-MOF-74,198.82,293,150.413


In [3]:
mw = Dict("CO2" => 44.01, "SO2" => 64.066)
for gas in ["CO2", "SO2"]
    df_henry[Symbol("H_$gas [g/(g-bar)]")] = df_henry[Symbol(gas * "_henry coefficient [mmol/(g-bar)]")] * mw[gas] / 1000
end

first(df_henry)

,crystal,CO2_henry coefficient [mmol/(g-bar)],CO2_Temperature [K],SO2_henry coefficient [mmol/(g-bar)]
,String⍰,Float64⍰,Int64⍰,Float64⍰
1,MFM601,1.3703,298,97.2407


In [4]:
construct_H_matrix(mofs::Array{Int64, 1}, gases::Array{<:AbstractString, 1}) = [df_henry[mof, Symbol("H_" * gas * " [g/(g-bar)]")] for mof in mofs, gas in gases]
construct_H_matrix([7, 9], ["SO2", "CO2"])


2×2 Array{Float64,2}:
 10.983    0.202777
  9.63634  8.75008 

In [16]:
# examines the 14,542 different combos of H2 and H3 matrices and compares δp2 to δp3
# 12C2 = 66 different combos for H2, 12C3 = 220 different combos for H3 ---> 220*66 = 14,520 

# δp3 > δp2
p3GTp2 = 0
for a=1:12
    for b=a+1:12
        H2 = construct_H_matrix([a,b], ["CO2", "SO2"])
        F2 = svd(H2)
        δp2 = 1 / F2.S[2] * sqrt(2)         # 1/F2.S[2] is the largest singluar value of H2^{-1}
        # construct 3x2 matrix named H3
        for i=1:12
            for j=i+1:12
                for k=j+1:12
                    H3 = construct_H_matrix([i,j,k], ["CO2", "SO2"])
                    F3 = svd(H3)
                    δp3 = 1 / F3.S[2] * sqrt(3)
                    # compare δp values
                    if δp3 > δp2
                        p3GTp2 += 1 
                        #println("δp3 is greater than δp2 for: ")
                        #println("H2: ", df_henry[:crystal][a], ", ", df_henry[:crystal][b])
                        #println("H3: ", df_henry[:crystal][i], ", ", df_henry[:crystal][j], ", ", df_henry[:crystal][k], "\n")
                    end
                end
            end
        end
    end
end
println("There were ", p3GTp2, " values of δp3 that were greater than δp2.")

There were 5223 values of δp3 that were greater than δp2.


In [17]:
# δp3 < δp2
p3LTp2 = 0
for a=1:12
    for b=a+1:12
        H2 = construct_H_matrix([a,b], ["CO2", "SO2"])
        F2 = svd(H2)
        δp2 = 1 / F2.S[2] * sqrt(2)         # 1/F2.S[2] is the largest singluar value of H2^{-1}
        # construct 3x2 matrix named H3
        for i=1:12
            for j=i+1:12
                for k=j+1:12
                    H3 = construct_H_matrix([i,j,k], ["CO2", "SO2"])
                    F3 = svd(H3)
                    δp3 = 1 / F3.S[2] * sqrt(3)
                    # compare δp values
                    if δp3 < δp2
                        p3LTp2 += 1 
                        #println("δp3 is less than δp2 for: ")
                        #println("H2: ", df_henry[:crystal][a], ", ", df_henry[:crystal][b])
                        #println("H3: ", df_henry[:crystal][i], ", ", df_henry[:crystal][j], ", ", df_henry[:crystal][k], "\n")
                    end
                end
            end
        end
    end
end
println("There were ", p3LTp2, " values of δp3 that were less than δp2.")

There were 9297 values of δp3 that were less than δp2.


In [8]:
# examines H matrices where the additional row on H3 is the MOF from row 2
# 792 possibilities total (66*12)
mof_count = zeros(12)
num = 0
for a=1:12
    for b=a+1:12
        H2 = construct_H_matrix([a,b], ["CO2", "SO2"])
        F2 = svd(H2)
        δp2 = 1 / F2.S[2] * sqrt(2)         # 1/F2.S[2] is the largest singluar value of H2^{-1}
        for c=1:12
            H3 = construct_H_matrix([a,b,c], ["CO2", "SO2"])
            F3 = svd(H3)
            δp3 = 1 / F3.S[2] * sqrt(3)
            # compare δp values
            if (δp3 > δp2)
                num += 1
                mof_count[c] += 1
                #println("δp3 is greater than δp2 for: ")
                #println("H2: ", df_henry[:crystal][a], ", ", df_henry[:crystal][b])
                #println("H3: ", df_henry[:crystal][a], ", ", df_henry[:crystal][b], ", ", df_henry[:crystal][c], "\n")
            end
        end
    end
end
println(num)
mof_count

330


12-element Array{Float64,1}:
 31.0
 31.0
 31.0
 35.0
 40.0
 28.0
 29.0
 26.0
 16.0
 31.0
 16.0
 16.0